# Load libraries and data

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import re
from pathlib import Path
from collections import Counter
import wandb

In [27]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: daniele-didino to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
train_data = pd.read_csv(Path("data", "processed", "train.csv"))
val_data = pd.read_csv(Path("data", "processed", "val.csv"))

# Parameters & wandb

In [28]:
MIN_FREQ = 1 # 20
MAX_LEN = 20
EMBED_DIM = 50
NUM_CLASSES = 6 # toxic, severe_toxic, obscene, threat, insult, identity_hate
BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using {DEVICE}")

Using cuda


In [29]:
run = wandb.init(
    # Set the project where this run will be logged
    project="toxic_comment_clf",
    # Track hyperparameters
    config={
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "embed_dim": EMBED_DIM
    },
)

In [30]:
run.config

{'learning_rate': 0.001, 'epochs': 1, 'batch_size': 32, 'embed_dim': 50}

# Tokenizer

In [4]:
# Prepare Tokenizer and util functions
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    return text


def build_vocab(texts: list[str], min_freq: int=1) -> dict:
    token_counts = Counter()
    for text in texts:
        cleaned_text = clean_text(text)
        token_counts.update(cleaned_text.split())
    vocab = {word: idx + 2 for idx, (word, count) in enumerate(token_counts.items()) if count >= min_freq}
    vocab['<PAD>'] = 0
    vocab['<UNK>'] = 1
    return vocab


def tokenizer(text: str, vocab: dict, max_len: int) -> dict:
    cleaned_text = clean_text(text)
    tokens = [vocab.get(word, 1) for word in cleaned_text.split()[:max_len]]
    input_ids = tokens + [0] * (max_len - len(tokens))

    # Check if token exceeds the len of the voceb
    for token in input_ids:
        if token >= len(vocab):
            print(f"Warning: Token index {token} out of range!")
    
    return {'input_ids': torch.tensor(input_ids)}

# Model

In [ ]:
# Dataset Class
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = torch.tensor(self.labels[index], dtype=torch.float32)
        encoded = self.tokenizer(text)
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'labels': label
        }


# Model
class ToxicityClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, input_ids):
        embedded = self.embedding(input_ids).mean(dim=1)
        return self.fc(embedded)


# Compute Loss and Metrics
def loss_metrics(model, dataloader, criterion, device, threshold=0.5):
    model.eval()  # set model to evaluation mode
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():  # No gradients during evaluation
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Apply sigmoid to convert logits to probabilities
            probs = torch.sigmoid(outputs)

            # Save predictions
            all_labels.append(labels.cpu())
            all_probs.append(probs.cpu())
            all_preds.append((probs >= threshold).int().cpu())

    # Concatenate results
    all_labels = torch.cat(all_labels).numpy()
    all_probs = torch.cat(all_probs).numpy()
    all_preds = torch.cat(all_preds).numpy()

    # Calculate average loss
    avg_loss = total_loss / len(dataloader)

    # Compute Metrics (macro-averaged for multi-label tasks)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    # AUC-ROC (for multi-label, compute per class and take average)
    auc_roc = roc_auc_score(all_labels, all_probs, average='macro')

    return avg_loss, accuracy, precision, recall, f1, auc_roc


# Training function
def train_model(model, dataloader, criterion, optimizer, epochs, device):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

In [ ]:
def train_with_val(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    train_history = {
        "train_loss": [],
        "train_loss_2": [],
        "train_loss": [],
        "train_accuracy": [],
        "train_precision": [],
        "train_recall": [],
        "train_f1": [],
        "train_auc_roc": [],
        "val_loss": [],
        "val_accuracy": [],
        "val_precision": [],
        "val_recall": [],
        "val_f1": [],
        "val_auc_roc": []
    }
    model.to(device)

    for epoch in range(epochs):
        model.train()  # set model to training mode
        total_train_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Evaluate on validation set
        train_loss = total_train_loss / len(train_loader)
        val_loss, val_acc, val_prec, val_rec, val_f1, val_auc_roc = loss_metrics(model, val_loader, criterion, device)
        train_loss_2, train_acc, train_prec, train_rec, train_f1, train_auc_roc = loss_metrics(model, train_loader, criterion, device)

        # train set
        train_history["train_loss"].append(train_loss)
        train_history["train_loss_2"].append(train_loss_2)
        train_history["train_accuracy"].append(train_acc)
        train_history["train_precision"].append(train_prec)
        train_history["train_recall"].append(train_rec)
        train_history["train_f1"].append(train_f1)
        train_history["train_auc_roc"].append(train_auc_roc)
        # Validation set
        train_history["val_loss"].append(val_loss)
        train_history["val_accuracy"].append(val_acc)
        train_history["val_precision"].append(val_prec)
        train_history["val_recall"].append(val_rec)
        train_history["val_f1"].append(val_f1)
        train_history["val_auc_roc"].append(val_auc_roc)

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Loss: {train_loss_2:.4f} | Accuracy: {train_acc:.4f} | AUC_ROC: {train_auc_roc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f} | AUC_ROC: {val_auc_roc:.4f}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_loss_2": train_loss_2,
            "train_accuracy": train_acc,
            "train_precision": train_prec,
            "train_recall": train_rec,
            "train_f1": train_f1,
            "train_auc_roc": train_auc_roc,
            "val_loss": val_loss,
            "val_accuracy": val_acc,
            "val_precision": val_prec,
            "val_recall": val_rec,
            "val_f1": val_f1,
            "val_auc_roc": val_auc_roc,
        })

    return train_history


# FIN QUI

In [18]:
train_input = train_data.comment_text.to_list()
train_labels = train_data.loc[:, ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

val_input = val_data.comment_text.to_list()
val_labels = val_data.loc[:,  ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

In [21]:
vocab = build_vocab(train_input, MIN_FREQ)

# Model with masked output

In [104]:
# Custom Loss with Masking
class HierarchicalBCELoss(nn.Module):
    def forward(self, outputs, labels):
        toxic_loss = nn.functional.binary_cross_entropy_with_logits(outputs[:, 0], labels[:, 0])

        # Mask sub-category losses if toxic == 0 (i.e., non-toxit text)
        mask = labels[:, 0] > 0 # Consider sub-categories only if toxic == 1 (i.e., toxic text)
        sub_loss = nn.functional.binary_cross_entropy_with_logits(outputs[:, 1:], labels[:, 1:], reduction='none')
        sub_loss = (sub_loss * mask.unsqueeze(1)).mean()

        return toxic_loss + sub_loss

In [105]:
# Prepare dataset
dataset = ToxicCommentsDataset(train_input, train_labels, lambda text: tokenizer(text, vocab, MAX_LEN), MAX_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [106]:
model_mo = ToxicityClassifier(vocab_size=len(vocab), embed_dim=EMBED_DIM, num_classes=NUM_CLASSES)
model_mo.to(DEVICE)

# Loss (custom)
criterion_mo = HierarchicalBCELoss()

# Optimizer
optimizer_mo = optim.Adam(model.parameters(), lr=0.001)

train_model(model_mo, dataloader, criterion_mo, optimizer_mo, EPOCHS, DEVICE)

KeyboardInterrupt: 

In [ ]:
# Post-processing predictions
def postprocess_predictions(outputs):
    outputs = torch.sigmoid(outputs)
    toxic_pred = outputs[:, 0] >= 0.5

    # Zero out sub-categories if not toxic
    outputs[:, 1:] *= toxic_pred.unsqueeze(1)
    return outputs